### First, load the provided branch file as well as create a dataframe for a simple example to validate functions

In [393]:
example = {'i': [2038, 2038, 5628, 5628, 5628, 2038, 2038], 
        'j': [8945, 5563, 8945, 5563, 745, 5628, 5628], 
        'ckt_id': [1, 2, 1, 1, 1, 1, 2]}
df_example = pd.DataFrame(example)
df_example.head(2)

,i,j,ckt_id
0,2038,8945,1
1,2038,5563,2


In [395]:
df_branch = pd.read_csv('branch.csv')
df_branch.head(2)

,i,j,ckt_id
0,1,3,22
1,1,300020,21


### compute_hops
Finds the minimum number of hops between two buses in the network

In [404]:
from collections import deque

def compute_hops(dataframe, start_bus, end_bus):
    # Build a graph from the dataframe
    graph = {}
    for _, row in dataframe.iterrows():
        bus_from = row['i']
        bus_to = row['j']

        # Checks if a given bus (from i or j) is already in the graph
        if bus_from not in graph:
            graph[bus_from] = set()
        if bus_to not in graph:
            graph[bus_to] = set()

        # Each bus connection is added bidirectionally because the connections are undirected 
        graph[bus_from].add(bus_to)
        graph[bus_to].add(bus_from)
    
    # Initialize a double-ended queue to keep track of the current bus and number of hops it's taken to reach it
    queue = deque([(start_bus, 0)])  # (current_bus, current_hops)

    # Create a set to keep track of buses already explored, to avoid repetition 
    visited = set()

    # Runs while there are still un-checked buses in the queue
    while queue:
        # Left item in the queue contains current bus and count of hops
        current_bus, hops = queue.popleft()

        if current_bus == end_bus:
            return hops # end if at target bus
        
        visited.add(current_bus) # mark as visited

        # add unvisited neighbors to the right of the double-ended queue
        for neighbor in graph[current_bus]:
            if neighbor not in visited:
                queue.append((neighbor, hops + 1))
    
    return -1  # Return -1 if there's no path between start_bus and end_bus

#### Validate that it works with a simple example

In [412]:
compute_hops(df_example, 745, 8945)

2

### buses_within_hops

Lists all buses within a given number of hops from a starting bus

In [427]:
def buses_within_hops(dataframe, start_bus, max_hops):
    # Same process of building the graph, initializing varialbes
    graph = {}
    for _, row in dataframe.iterrows():
        bus_from = row['i']
        bus_to = row['j']
        
        if bus_from not in graph:
            graph[bus_from] = set()
        if bus_to not in graph:
            graph[bus_to] = set()
        
        graph[bus_from].add(bus_to)
        graph[bus_to].add(bus_from)
    
    queue = deque([(start_bus, 0)])
    visited = set([start_bus])

    # Create a set to keep track of all 
    result = set()
    
    while queue:
        current_bus, hops = queue.popleft()

        # stop searching when maximum number of hops is reached
        if hops > max_hops:
            break

        # append found buses
        result.add(current_bus)
        
        for neighbor in graph[current_bus]:
            if neighbor not in visited:
                visited.add(neighbor)
                queue.append((neighbor, hops + 1))

    # remove the starting bus and return result
    return result - {start_bus}

In [429]:
buses_within_hops(df_example, 8945, 1)

{2038, 5628}

In [431]:
buses_within_hops(df_branch, 3, 4)

{1,
 2,
 300020,
 300021,
 300024,
 300033,
 300069,
 300739,
 300740,
 301348,
 512626,
 512629,
 512635,
 512638,
 512648,
 512656,
 512760,
 513050}